In [ ]:
import cv2
import numpy as np

# 获取所有可用摄像头
num_cameras = 10  # 假设最多有10个摄像头
available_cameras = []
for i in range(num_cameras):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        available_cameras.append(i)
        cap.release()

# 可用摄像头列表
print("可用摄像头:")
for idx in available_cameras:
    print(f"摄像头 {idx}")
# 选择
selected_camera = int(input("请输入要使用的摄像头编号: "))
# 打开
video = cv2.VideoCapture(selected_camera)
# 检查
if video.isOpened():
    print(f"摄像头 {selected_camera} 已成功打开。")
else:
    print(f"无法打开选择的摄像头（摄像头 {selected_camera}）。请检查摄像头是否连接正常。")
    exit()

In [ ]:
import serial.tools.list_ports 
import struct 
import re  # 导入正则表达式模块

while True:

    # 从摄像头读取一帧
    ret, img = video.read()

    # 如果读取失败，退出循环
    if not ret:
        break

    # 将图像从BGR转换为HSV颜色空间
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # 定义蓝色的HSV范围
    lower_blue = np.array([100, 50, 50])
    upper_blue = np.array([130, 255, 255])

    # 创建蓝色的掩码
    blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # 对掩码应用高斯模糊
    blue_mask = cv2.GaussianBlur(blue_mask, (5, 5), 0)

    # 获取二值图像的尺寸
    whole_h, _ = blue_mask.shape[:2]

    # 在蓝色掩码中查找轮廓
    _,contours, _ = cv2.findContours(image=blue_mask, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)

    # 将轮廓转换为列表并按面积降序排列
    contours = sorted(contours, key=lambda c: cv2.contourArea(c), reverse=True)[:5]

    # 存储矩形宽度、高度和点的数组
    rects = []

    # 遍历前5个轮廓
    for cont in contours:
        # 获取边界矩形的坐标和尺寸
        x, y, w, h = cv2.boundingRect(cont)

        # 检查有效矩形的条件
        if h / w >= 2 and h / whole_h > 0.1 and h > w:
            rects.append([x, y, w, h])

    # 找到面积最接近的两个矩形
    min_value = float('inf')
    best_rects = []

    for i in range(len(rects) - 1):
        for j in range(i + 1, len(rects)):
            value = abs(rects[i][2] * rects[i][3] - rects[j][2] * rects[j][3])
            if value < min_value:
                min_value = value
                best_rects = [rects[i], rects[j]]

    # 如果找到了两个最近矩形
    if best_rects:
        # 计算并打印矩形的四个角点
        rectangle1, rectangle2 = best_rects
        point1 = [rectangle1[0] + rectangle1[2] / 2, rectangle1[1]]
        point2 = [rectangle1[0] + rectangle1[2] / 2, rectangle1[1] + rectangle1[3]]
        point3 = [rectangle2[0] + rectangle2[2] / 2, rectangle2[1]]
        point4 = [rectangle2[0] + rectangle2[2] / 2, rectangle2[1] + rectangle2[3]]

         # 两边两个点的中间坐标
        center_point1 = [(point1[0] + point2[0]) / 2, (point1[1] + point2[1]) / 2]
        center_point2 = [(point3[0] + point4[0]) / 2, (point3[1] + point4[1]) / 2]
        #中心坐标
        middle_point = [(center_point1[0] + center_point2[0]) / 2, (center_point1[1] + center_point2[1]) / 2]

        #print(point1, point2, point3, point4)
        #print("Center Points:", center_point1, center_point2)
        print("Middle Point:", middle_point)

        # 定义矩形为多边形并在原始图像上绘制它
        pts = np.array([point1, point2, point4, point3], np.int32).reshape((-1, 1, 2))
        cv2.polylines(img, [pts], isClosed=True, color=(0, 255, 0), thickness=2)

        #cv2.circle(img, (int(center_point1[0]), int(center_point1[1])), radius=5, color=(255, 0, 0), thickness=-1)
        #cv2.circle(img, (int(center_point2[0]), int(center_point2[1])), radius=5, color=(0, 0, 255), thickness=-1)
        cv2.circle(img, (int(middle_point[0]), int(middle_point[1])), radius=5, color=(0, 255, 255), thickness=-1)

        # 获取可用的串口列表
        available_ports = list(serial.tools.list_ports.comports())
        # 打开串口
        # 如果有可用的串口，则打开第一个串口
        if available_ports:
            port = available_ports[0][0]
            ser = serial.Serial(port, 9600)
        
        # 提取百位
            hex_middle_point_0 = int(middle_point[0] // 100)
            if hex_middle_point_0 < 10:
                hex_middle_point_0 = f'0{hex_middle_point_0}'
            else:
                hex_middle_point_0 = str(hex_middle_point_0)
        # 提取十位和个位
            hex_middle_point_1 = int(middle_point[0] % 100)
            if hex_middle_point_1 < 10:
                hex_middle_point_1 = f'0{hex_middle_point_1}'
            else:
                hex_middle_point_1 = str(hex_middle_point_1)
        # 提取小数点后两位
            hex_middle_point_2 = int((middle_point[0] % 1) * 100)
        # 提取百位以上2
            hex_middle_point_3 = int(middle_point[1] // 100)
            if hex_middle_point_3 < 10:
                hex_middle_point_3 = f'0{hex_middle_point_3}'
            else:
                hex_middle_point_3 = str(hex_middle_point_3)
        # 提取十位和个位2
            hex_middle_point_4 = int(middle_point[1] % 100)
            if hex_middle_point_4 < 10:
                hex_middle_point_4 = f'0{hex_middle_point_4}'
            else:
                hex_middle_point_4 = str(hex_middle_point_4)
        # 提取小数点后两位2
            hex_middle_point_5 = int((middle_point[1] % 1) * 100)
            
            hex_string = f'0x0A 0x0B 0x{hex_middle_point_0} 0x{hex_middle_point_1} 0x{hex_middle_point_2} 0x{hex_middle_point_3} 0x{hex_middle_point_4} 0x{hex_middle_point_5} 0x0C 0x0D'
        
            print("Hex String:", hex_string)
        # 发送数据到串口
            byte_data = hex_string.encode('utf-8')
            # 打开串口
            #ser = serial.Serial(port, baudrate=9600, timeout=1)
            try:
                # 将字节写入串口
                ser.write(byte_data)
                print("成功发送字符串到USB。")
            except Exception as e:
                print(f"发送字符串到USB时发生错误：{e}")
            finally:
            # 发送换行符
                ser.write(b'\n')
            # 关闭串口
                ser.close()
        else:
            print("未找到可用的 COM 端口。")

    # 显示摄像头画面
    cv2.imshow('Camera Feed', img)

    # 检查是否按下 'q' 键以退出循环
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
#释放摄像头资源和关闭窗口
video.release() 
cv2.destroyAllWindows()